# **fGraph 0 detection with Decision Trees and SVM from eigenvalues**


<div id="import_libraries">
    <h2>Import Libraries</h2>
</div>


In [1]:
#!pip install pandas
#!pip install scikit-learn
#!pip install matplotlib
#!pip install xgboost

To import the libraries that will be used in this lab, execute the cells below. 


In [1]:
# Import the libraries we need to use in this lab
from __future__ import print_function
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import ast
import time
# model trainin set
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import normalize, StandardScaler
from sklearn.utils.class_weight import compute_sample_weight
#models 
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import LinearSVC
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBRegressor
#model evaluation
from sklearn.metrics import roc_auc_score
from sklearn.metrics import mean_squared_error, r2_score

## Load the dataset




In [4]:
# load the dataset
path= "fgraph_eigen_8.csv"
# read the input data
raw_data = pd.read_csv(path)
raw_data


,COEFFICIENT_DEN,DEN_EIGEN1,DEN_EIGEN2,DEN_EIGEN3,DEN_EIGEN4,DEN_EIGEN5,DEN_EIGEN6,DEN_EIGEN7,DEN_EIGEN8,DEN_EIGEN9,...,FGRAPH_EIGEN3,FGRAPH_EIGEN4,FGRAPH_EIGEN5,FGRAPH_EIGEN6,FGRAPH_EIGEN7,FGRAPH_EIGEN8,FGRAPH_EIGEN9,FGRAPH_EIGEN10,FGRAPH_EIGEN11,FGRAPH_EIGEN12
0,0,-2.472834,-2.156712,-2.000000,-1.462598,-1.319479,-1.000000,-7.151720e-01,-4.142136e-01,1.935432,...,-2.000000,-2.000000,-1.667742,-7.320508e-01,-2.267388e-01,-1.332268e-15,2.732051e+00,2.828427,3.200059,4.0
1,0,-2.472834,-2.156712,-2.000000,-1.462598,-1.319479,-1.000000,-7.151720e-01,-4.142136e-01,1.935432,...,-2.445281,-2.000000,-1.287292,-8.402523e-01,-3.974333e-01,-7.177127e-02,2.617707e+00,3.042355,3.197411,4.0
2,0,-2.472834,-2.156712,-2.000000,-1.462598,-1.319479,-1.000000,-7.151720e-01,-4.142136e-01,1.935432,...,-2.000000,-2.000000,-1.538445,-1.236068e+00,-1.236068e+00,-2.968418e-01,2.878345e+00,3.236068,3.236068,4.0
3,0,-2.472834,-2.156712,-2.000000,-1.462598,-1.319479,-1.000000,-7.151720e-01,-4.142136e-01,1.935432,...,-2.000000,-2.000000,-1.511407,-1.236068e+00,-5.364049e-01,-3.576331e-02,2.802365e+00,3.012160,3.236068,4.0
4,0,-2.472834,-2.156712,-2.000000,-1.462598,-1.319479,-1.000000,-7.151720e-01,-4.142136e-01,1.935432,...,-2.000000,-1.791997,-1.591352,-1.358032e+00,-3.484352e-01,9.251150e-02,2.567288e+00,3.096807,3.245414,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2704,0,-2.618034,-2.618034,-1.618034,-1.618034,-0.561553,-0.381966,-3.819660e-01,4.384472e-01,0.618034,...,-1.456850,-1.456850,-0.543150,-5.431497e-01,-1.372714e-16,9.969958e-16,1.188901e+00,1.188901,4.000000,4.0
2705,0,-2.900523,-2.000000,-2.000000,-1.581807,-0.798880,-0.486482,4.346060e-16,1.341751e-15,0.591077,...,-1.618034,-1.302776,-0.697224,-3.819660e-01,-7.036912e-16,9.482206e-16,6.180340e-01,2.302776,4.000000,4.0
2706,0,-2.618034,-2.230725,-1.874076,-1.618034,-0.482859,-0.381966,-1.519943e-16,1.519943e-16,0.486456,...,-2.000000,-1.414214,-0.585786,-1.599715e-15,-2.573573e-16,2.915845e-17,1.168644e-16,1.414214,4.000000,4.0
2707,0,-2.618034,-2.230725,-1.874076,-1.618034,-0.482859,-0.381966,-1.519943e-16,1.519943e-16,0.486456,...,-2.000000,-1.414214,-0.585786,-1.599715e-15,-2.573573e-16,2.915845e-17,1.168644e-16,1.414214,4.000000,4.0


In [112]:
# Assuming the first column is 'COEFFICIENT_DEN'
cols_to_round = raw_data.columns.difference(['COEFFICIENT_DEN'])

# Round those columns to, say, 6 decimals
raw_data[cols_to_round] = df[cols_to_round].round(6)
raw_data

,COEFFICIENT_DEN,EV_DEN1,EV_DEN2,EV_DEN3,EV_DEN4,EV_DEN5,EV_DEN6,EV_DEN7,EV_DEN8,EV_DEN9,...,CYCLES_MULT4,CYCLES_MULT5,CYCLES_MULT6,CYCLES_MULT7,CYCLES_MULT8,CYCLES_MULT9,CYCLES_MULT10,CYCLES_MULT11,CONNECTIVITY,DOUBLE_TRIANGLES
0,1,-2.2258,-1.7321,-1.7321,-1.000,-1.0000,-0.7259,1.7321,1.7321,2.2361,...,29,78,234,566,1018,1322,1152,508,4,18
1,1,-2.2258,-1.7321,-1.7321,-1.000,-1.0000,-0.7259,1.7321,1.7321,2.2361,...,29,78,234,566,1018,1322,1152,508,4,18
2,1,-2.2258,-1.7321,-1.7321,-1.000,-1.0000,-0.7259,1.7321,1.7321,2.2361,...,29,78,234,566,1018,1322,1152,508,4,18
3,1,-2.2258,-1.7321,-1.7321,-1.000,-1.0000,-0.7259,1.7321,1.7321,2.2361,...,29,78,234,566,1018,1322,1152,508,4,18
4,1,-2.2258,-1.7321,-1.7321,-1.000,-1.0000,-0.7259,1.7321,1.7321,2.2361,...,29,78,234,566,1018,1322,1152,508,4,18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
215,0,-2.1966,-1.7554,-1.6573,-1.000,-0.4816,-0.2213,1.1938,1.5071,2.4820,...,23,57,134,260,415,520,441,178,4,5
216,0,-2.0000,-1.7524,-1.5616,-1.000,-0.3074,-0.0000,1.0000,1.0362,2.5616,...,34,86,175,296,414,488,406,156,3,9
217,0,-2.2709,-2.0000,-1.4652,-1.000,-0.1802,0.0000,1.0000,1.2527,2.6636,...,19,48,94,160,243,308,259,102,3,5
218,0,-2.0000,-1.7457,-1.5456,-0.647,-0.4240,0.0000,0.3520,1.4172,2.8405,...,27,57,107,208,369,507,453,183,3,9


# ChatGpt clasifier computation

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score

# 1. Load dataset
df = raw_data

# 2. Extract features and target
y = df["COEFFICIENT_DEN"]
X = df.drop(columns=["COEFFICIENT_DEN"])


ValueError: The least populated class in y has only 1 member, which is too few. The minimum number of groups for any class cannot be less than 2.

In [ ]:


# 3. Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# 4. Scale the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# 5. Train a classifier (Logistic Regression here)
clf = LogisticRegression(max_iter=1000)
clf.fit(X_train_scaled, y_train)

# 6. Predict and evaluate
y_pred = clf.predict(X_test_scaled)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))


### most important feature

In [118]:
# 7. Get feature importance from Logistic Regression
feature_names = X.columns
coefficients = clf.coef_[0]  # For binary classification, it's a 1D array

# Create a DataFrame for sorting and readability
feature_importance = pd.DataFrame({
    'feature': feature_names,
    'coefficient': coefficients,
    'abs_coefficient': np.abs(coefficients)
})

# Sort by absolute value of coefficients
feature_importance_sorted = feature_importance.sort_values(by='abs_coefficient', ascending=False)

# Display the most relevant feature
most_relevant = feature_importance_sorted.iloc[0:5]
print("Most relevant features:")
print(most_relevant)


Most relevant features:
             feature  coefficient  abs_coefficient
8            EV_DEN9    -0.948758         0.948758
2            EV_DEN3     0.691359         0.691359
4            EV_DEN5     0.528816         0.528816
20       DEN_WALKS11     0.516590         0.516590
31  DOUBLE_TRIANGLES     0.513317         0.513317


### Decision tree

In [119]:

# 4. (Optional) scale features – not needed for decision trees but harmless
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# 5. Train a Decision Tree classifier
tree_clf = DecisionTreeClassifier(random_state=42)
tree_clf.fit(X_train_scaled, y_train)

# 6. Predict and evaluate
y_pred = tree_clf.predict(X_test_scaled)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

Accuracy: 0.8181818181818182
Classification Report:
               precision    recall  f1-score   support

           0       0.65      0.72      0.68        18
           1       0.89      0.85      0.87        48

    accuracy                           0.82        66
   macro avg       0.77      0.79      0.78        66
weighted avg       0.83      0.82      0.82        66



In [ ]:
# 7. Feature importances from the decision tree
importances = tree_clf.feature_importances_
feature_importance = pd.DataFrame({
    'feature': X.columns,
    'importance': importances
}).sort_values(by='importance', ascending=False)

print("Most important features:")
print(feature_importance.iloc[0:4])

Most important features:
       feature  importance
8      EV_DEN9    0.248889
4      EV_DEN5    0.113565
5      EV_DEN6    0.101068
13  DEN_WALKS4    0.087601


### Random forest

In [121]:
from sklearn.ensemble import RandomForestClassifier

# 1. Train the Random Forest
rf_clf = RandomForestClassifier(n_estimators=100, random_state=42)
rf_clf.fit(X_train_scaled, y_train)

# 2. Predict and evaluate
y_pred_rf = rf_clf.predict(X_test_scaled)

print("Random Forest Accuracy:", accuracy_score(y_test, y_pred_rf))
print("Classification Report:\n", classification_report(y_test, y_pred_rf))

Random Forest Accuracy: 0.8939393939393939
Classification Report:
               precision    recall  f1-score   support

           0       0.87      0.72      0.79        18
           1       0.90      0.96      0.93        48

    accuracy                           0.89        66
   macro avg       0.88      0.84      0.86        66
weighted avg       0.89      0.89      0.89        66



In [122]:
# 7. Feature importances from the decision tree
importances = rf.feature_importances_
feature_importance = pd.DataFrame({
    'feature': X.columns,
    'importance': importances
}).sort_values(by='importance', ascending=False)

print("Most important features:")
print(feature_importance.iloc[0:4])

Most important features:
         feature  importance
8        EV_DEN9    0.240560
7        EV_DEN8    0.053966
30  CONNECTIVITY    0.047965
3        EV_DEN4    0.042049
